# Group 6 Term Project Implementation
- Yu Pan
- Benjamin Wechsler 
- Chloe Wang 
- Xinyu Wu 
- Yaoze Liu 

Note that the original code was written in three modules: 
1. _news_etl.py_: this module creates a data pipeline that pulls, transforms and load data into the data base on Bigquery. 
2. _query.py_: this module contains the query strings and make requests through Bigquery API
3. _app.py_: this module contains the main body of the web application, when the search() dunction is called, the news data ETL pipeline is activated and two queries(one query for the FIFA stats data and one for the news data)

For the ease of your grading, the above modules are combined here and the grading points are specified below: 
- Design of the back-end data warehouse(s): see _news_etl.py_ and _query.py_
- Data ETL process pipeline: see _news_etl.py_
- Interface to the designed system: see _app.py_
- Inclusion of at least 3 technologies studied in the course: 
  - Google cloud as data warehouse
  - Bigquery for relational database 
  - Flask for user interface


First, define the functions required for the News data pipeline. This is originally from _news_etl.py_

In [1]:
from newsapi import NewsApiClient
from google.cloud import bigquery
import os

# read API key and establish a newsapi client
with open("newsapi_key.txt", "r") as file: 
  api_key = file.read().strip()
file.close()
newsapi = NewsApiClient(api_key=api_key)

# establish a bigquery client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'cloud_service_account.json'
client = bigquery.Client()

# Extract data from the News API 
def extract_data(query: str, language="en", page_size=3, max_pages=2):
    # this function takes in a string, and send the string to newsapi
    # to request 3(default) articles
    all_articles = []
    page = 1
    while True:
        articles = newsapi.get_everything(q=query, language=language, page_size=page_size, page=page)
        if not articles['articles'] or page >= max_pages:
            break
        all_articles.extend(articles['articles'])
        page += 1
    return all_articles


# Transform the data into a suitable format
def transform_data(articles, search_query):
    # this function takes in a list of articles generated by extract_data() and the 
    # query as the key word to transform the data into a list of json object to be
    # loaded into big query. This way the data can match our schema in the database
    transformed_articles = []
    for article in articles:
        transformed_articles.append(
            {
                "key_word": search_query,
                "source_id": article["source"]["id"],
                "source_name": article["source"]["name"],
                "author": article["author"],
                "title": article["title"],
                "description": article["description"],
                "url": article["url"],
                "url_to_image": article["urlToImage"],
                "published_at": article["publishedAt"],
                "content": article["content"],
            }
        )
    return transformed_articles

# loading the data into bigquery
def load_data_to_bigquery(transformed_articles, dataset_id, table_id):
    # This function uploads the transformed data to bigquery

    # Load the data into the table
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)

    errors = client.insert_rows_json(table, transformed_articles)
    if errors:
        raise RuntimeError("Failed to load data to BigQuery: {}".format(errors))


Next, write the methods required for the search function and interaction with the data tables. This is originally from _query.py_

In [2]:
def query_by_player(name:str, info: list): 
  # the function takes in a name and search match it on bigquery. the info argument
  # is a list of extra info that the user would like to see by checking the boxes
  # in the web interface
  
  # convert info from list to string to append it into the query
  info = ", " + ", ".join(info)

  query = f"""
  SELECT DISTINCT fifa_version, long_name, age, value_eur, club_name, nationality_name, player_face_url {info}
  FROM aerobic-goal-384015.soccer_player.player
  WHERE LOWER(long_name) LIKE LOWER('%{name}%') 
  ORDER BY fifa_version DESC
  """

  # run the query and return it as a dataframe
  query_job = client.query(query)
  return query_job.result().to_dataframe()

def query_by_coach(name: str): 
  # takes in a coach's name and query it from the coach data set
  
  query = f"""
  SELECT DISTINCT *
  FROM aerobic-goal-384015.soccer_player.coach
  WHERE LOWER(long_name) LIKE LOWER('%{name}%') 
  """

  # run the query and return it as a dataframe
  query_job = client.query(query)
  return query_job.result().to_dataframe()


def query_by_team(team: str): 
  # takes in a team's name and query it from the team data set
  
  query = f"""
  SELECT DISTINCT fifa_version, team_name, league_name, overall, coach_id
  FROM aerobic-goal-384015.soccer_player.team
  WHERE LOWER(team_name) LIKE LOWER('%{team}%')
  ORDER BY fifa_version DESC
  """

  # run the query and return it as a dataframe
  query_job = client.query(query)
  return query_job.result().to_dataframe()


def query_by_rating(rating): 
  # takes in a rating and query it from the player data set
  
  query = f"""
  SELECT DISTINCT fifa_version, long_name, age, value_eur, club_name, nationality_name, player_face_url
  FROM aerobic-goal-384015.soccer_player.player
  WHERE overall >= {rating}
  ORDER BY fifa_version DESC
  """

  # run the query and return it as a dataframe
  query_job = client.query(query)
  return query_job.result().to_dataframe()

def query_news(name): 
  # takes in any topic and query it from the news data set
  
  query = f"""
  SELECT DISTINCT key_word, source_name, title, description, content, url, published_at
  FROM aerobic-goal-384015.soccer_player.news
  WHERE key_word = '{name}'
  ORDER BY published_at DESC
  """

  # run the query and return it as a dataframe
  query_job = client.query(query)
  return query_job.result().to_dataframe()

Lastly, create the web application with Flask. This is originally from _app.py_

In [ ]:
from flask import Flask, render_template, request
#import query
#import news_etl
import cachetools

app = Flask(__name__)

# Set up a cache with a TTL of 12 hours and a maximum of 100 items
# because the API is limited to 100 requests per 12 hours
cache = cachetools.TTLCache(maxsize=100, ttl=12 * 60 * 60)

@app.route('/')
def index():
    return render_template('search.html')

@app.route('/search', methods=['POST'])
def search():
    search_query = request.form.get('search_query')
    search_by = request.form.get('search_by')
    info = request.form.getlist('info')
    
    if search_by == "player": 
        df = query_by_player(search_query,info)
    if search_by == "coach": 
        df = query_by_coach(search_query)
    if search_by == "team": 
        df = query_by_team(search_query)
    if search_by == "rating": 
        df = query_by_rating(search_query)

    table_html = df.to_html(escape=False, formatters={
        'player_face_url': lambda url: f'<img src="{url}" width="100" height="100" alt="Player Face">'}, index=False)

    # Check if the articles for the search_query are cached, otherwise fetch and cache them
    if search_query not in cache:
        articles = extract_data(search_query)
        transformed_articles = transform_data(articles, search_query)
        load_data_to_bigquery(transformed_articles, "soccer_player", "news")
        cache[search_query] = transformed_articles
    
    # query the just-loaded news data from the cloud, then display them on our website
    news = query_news(search_query).to_html() 

    return "Search results for: {} (search by: {})".format(search_query, search_by) + table_html + "Latest news about {}".format(search_query) + news

if __name__ == '__main__':
    app.run(debug=True)
